In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
!ls 'gdrive/My Drive'

 549_data.zip	   'How to get started with Drive.pdf'
 bert_models	    model.ckpt-0.data-00000-of-00001
'Colab Notebooks'   model.ckpt-500.data-00000-of-00001


In [3]:
!unzip -n 'gdrive/My Drive/549_data.zip'

Archive:  gdrive/My Drive/549_data.zip
  inflating: test.csv                
  inflating: train.csv               


In [4]:
!ls

gdrive	sample_data  test.csv  train.csv


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
from sklearn.metrics import classification_report


import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras
import os
import re


In [6]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 5.3MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
OUTPUT_DIR = 'outputs'

tf.gfile.MakeDirs(OUTPUT_DIR)



In [10]:
!ls

gdrive	outputs  sample_data  test.csv	train.csv


#Data

In [0]:
train = pd.read_csv('train.csv',sep='\t')
test = pd.read_csv('test.csv',sep='\t')

In [12]:
print(train.shape, test.shape)
print(train.head())
print(test.head())

(8336, 2) (6126, 2)
                                                text  label
0  @zj_marines that's good girl' I'm giving lily ...      3
1      I am going to need a Xanax after this #debate      2
2   My back freaking hurts wish I could pop a Xanax       2
3  Im miserably tired but can't sleep and still w...      1
4  the amount of ibuprofen i take is absolutely h...      2
                                                text  label
0  Sooooo I just realized my baby haven't been sl...      3
1  @caitbateman1 hasn't mentioned it once. Won't ...      3
2  My mouth is really irritating me , need some i...      2
3                             That clonazepam got me      1
4                       ibuprofen, hot tea & a bath       3


In [0]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

label_list = [1, 2, 3]


- `text_a` is the input
- `text_b` is used for translation or similar tasks. 
- `label` is the label... 


In [0]:


# all training data
all_train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


train_part , validation_part = train_test_split(train, test_size=0.1, random_state=42)



train_InputExamples = train_part.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)



validation_InputExamples = validation_part.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)



test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [15]:
all_train_InputExamples.shape,  train_InputExamples.shape,  validation_InputExamples.shape, test_InputExamples.shape 

((8336,), (7502,), (834,), (6126,))

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def build_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = build_tokenizer_from_hub_module()


These below are needed for Bert

In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 50
# Convert our train and test features to InputFeatures that BERT understands.

all_train_features = bert.run_classifier.convert_examples_to_features(all_train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
validation_features = bert.run_classifier.convert_examples_to_features(validation_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)




In [18]:
len(all_train_features), len(train_features),  len(validation_features), len(test_features)

(8336, 7502, 834, 6126)

#Getting a model

Load Bert from tf hub and add a layer with 3 units for classification.
Then fine tune the model.

In [0]:
def make_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
      # trainable=False)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # instantiate our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually makes our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = make_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        # f1_score = tf.contrib.metrics.f1_score(
        #     label_ids,
        #     predicted_labels)
        # auc = tf.metrics.auc(
        #     label_ids,
        #     predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            # "f1_score": f1_score,
            # "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = make_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size

BATCH_SIZE = 128
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 10.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


In [0]:
# input function for training.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Must use GPUs to train.

In [29]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Training took time  0:07:52.533445


Run Validation... cross val. is too expensive.

In [0]:
validation_input_fn = run_classifier.input_fn_builder(
    features=validation_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [31]:
res = estimator.evaluate(input_fn=validation_input_fn, steps=None)
for r in res:
    print(r,' : ', res[r])

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


eval_accuracy  :  0.8189449
false_negatives  :  35.0
false_positives  :  48.0
loss  :  0.9372541
precision  :  0.9296188
recall  :  0.9476831
true_negatives  :  117.0
true_positives  :  634.0
global_step  :  586


In [0]:
def getPrediction(in_sentences):
  labels = [1,2,3]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 1) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
y_true = validation_part['label']
preds = getPrediction(validation_part['text'])
y_pred = [x[2] for x in preds]

In [34]:
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

           1     0.7763    0.7152    0.7445       165
           2     0.7207    0.7917    0.7545       264
           3     0.9133    0.8840    0.8984       405

    accuracy                         0.8213       834
   macro avg     0.8034    0.7969    0.7991       834
weighted avg     0.8252    0.8213    0.8224       834



Once I determined my parameters, I train the model with all the training data and then use the test set.

In [35]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(all_train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

# Input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=all_train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)


print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Training took time  0:01:28.740454


In [0]:
y_true = test['label']
preds = getPrediction(test['text'])
y_pred = [x[2] for x in preds]

In [37]:
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

           1     0.6963    0.7906    0.7404      1409
           2     0.7713    0.6608    0.7118      2205
           3     0.8297    0.8710    0.8499      2512

    accuracy                         0.7769      6126
   macro avg     0.7658    0.7741    0.7674      6126
weighted avg     0.7780    0.7769    0.7750      6126



In [38]:
!ls outputs/

checkpoint				     model.ckpt-500.index
eval					     model.ckpt-500.meta
events.out.tfevents.1577614500.5828f8ef0782  model.ckpt-586.data-00000-of-00001
graph.pbtxt				     model.ckpt-586.index
model.ckpt-0.data-00000-of-00001	     model.ckpt-586.meta
model.ckpt-0.index			     model.ckpt-651.data-00000-of-00001
model.ckpt-0.meta			     model.ckpt-651.index
model.ckpt-500.data-00000-of-00001	     model.ckpt-651.meta
